## Config and Imports

In [1]:
model_path = "./ckpt/FFHQ_256.pt"
device = "cuda"
image_size = 256
image_width, image_height, rgb_channel, rgba_channel = 256, 256, 3, 4

In [2]:
import torch
from torchvision.transforms import functional
import os
from models.generator import *
import math
import cv2
import numpy as np
import matplotlib.pyplot as plt
from array import array
from draggan import DragGAN
import lpips
from einops import rearrange
import torch.nn as nn
import torch.nn.functional as F
from dift_sd import SDFeaturizer

print(torch.cuda.is_available())
os.add_dll_directory("C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v11.7\\bin")

True


C:\Users\PublicF\anaconda3\envs\stylegan3\lib\site-packages\diffusers\utils\outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


<AddedDllDirectory('C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v11.7\\bin')>

## Setup model

In [3]:
model = DragGAN(device)
model.load_ckpt(model_path)

# raw data
raw_data_size = image_width * image_height * rgba_channel
raw_data = array('f', [1] * raw_data_size)
origin_image = array('f', [1] * raw_data_size)

C:\Users\PublicF\anaconda3\envs\stylegan3\lib\site-packages\torch\functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


loading checkpoint from ./ckpt/LSUNChurch_256.pt
input size:  256
change generator to 256 weights
loading checkpoint successed!


## Generate points and masks

we don't neeed to "draw" any more so just return the point array

In [10]:
# points parameters
points = [np.array([120, 150]), np.array([105, 150])]
ptn = 2

# mask parameters
mask = np.ones([256, 256], np.uint8)

# seed
seeds = [156,251,653,1896,2534]

# overlay
overlays = [0, 1]

# record class
# overlay = [0, 1]
# slayer = [6, 7]
# sll = range(1,9)
class Record:
    def __init__(self, seed, overlay, layer, sll):
        self.seed = seed
        self.overlay = overlay
        self.layer = layer
        self.sll = sll
    def __str__(self):
        return "seed: " + str(self.seed) + ", overlay: " + str(self.overlay) + ", layer: " + str(self.layer) + ", sll: " + str(self.sll)

# records = [Record(0,0,0,0)]
records = []

seed: 0, overlay: 0, layer: 0


## Step & Score

1. generate image with different seed
2. while dragging, do thresholding and update the mask
3. calculate scores at the end

In [7]:
# functions
def update_image(new_image):
    for i in range(0, 256*256):
        rd_base, im_base = i * rgba_channel, i * rgb_channel
        raw_data[rd_base:rd_base + rgb_channel] = array('f', new_image[im_base:im_base + rgb_channel])


for seed in seeds:
    for overlay in overlays:
        # generate image and origin data
        image = model.generate_image(seed)
        update_image(image)
        for i in range(0, 256*256):
            rd_base, im_base = i * rgba_channel, i * rgb_channel
            origin_image[rd_base:rd_base + rgb_channel] = array('f', image[im_base:im_base + rgb_channel])
    
    
        # dragging parameters
        step = 0
        dragging = True
        
        # dragging
        # we need to automate different layer selection 
        while (dragging):
            status, ret, sfeatures, simage = model.step(points, mask, overlay)
            if status:
                points, image = ret
            else:
                dragging = False
                continue
            update_image(image)
            # update_mask(mask)
            step += 1
            
        # call calculate score function after dragging
        lpips = model.lpip_score(origin_image, raw_data)
        mds = model.mean_distance_score(origin_image, raw_data, points)
        print("lpips:", lpips)
        print("mds:", mds)
        # fixed layer to 6 for testing
        # same with draggan.py line 246
        records.append(Record(seed, overlay, 6, 4))


z:  torch.Size([1, 512])
p0:  [(120, 150)]
tensor(7.0829, device='cuda:0', grad_fn=<AddBackward0>)
tensor distance:  tensor([15.])
tensor(7.4676, device='cuda:0', grad_fn=<AddBackward0>)
tensor distance:  tensor([15.])
tensor(7.4715, device='cuda:0', grad_fn=<AddBackward0>)
tensor distance:  tensor([15.])
tensor(7.4484, device='cuda:0', grad_fn=<AddBackward0>)
tensor distance:  tensor([15.])
tensor(7.4060, device='cuda:0', grad_fn=<AddBackward0>)
tensor distance:  tensor([15.])
tensor(7.3771, device='cuda:0', grad_fn=<AddBackward0>)
tensor distance:  tensor([15.])
tensor(7.3310, device='cuda:0', grad_fn=<AddBackward0>)
tensor distance:  tensor([15.])
tensor(7.3043, device='cuda:0', grad_fn=<AddBackward0>)
tensor distance:  tensor([15.])
tensor(7.3122, device='cuda:0', grad_fn=<AddBackward0>)
tensor distance:  tensor([15.])
tensor(7.3258, device='cuda:0', grad_fn=<AddBackward0>)
tensor distance:  tensor([15.])
tensor(7.3308, device='cuda:0', grad_fn=<AddBackward0>)
tensor distance:  ten

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(20,55))

im_w, im_h = 256, 256
points = [np.array([120, 150]), np.array([90, 150])]
ptn = 2

extract_layers = [6,7]

for el in extract_layers:
    d = features[0][el].size(dim=1)
    d2 = features[0][el].size(dim=2)
    feature = features[0][el].transpose(-1,-2).reshape(1, d2, int(math.sqrt(d)), int(math.sqrt(d)))

    img = feature[0]
    if el == 6:
        sixthlayer = img.cpu().numpy()
    if el == 7:
        seventhlayer = img.cpu().numpy()

    for k in range(1,9):
        
        temp = np.zeros((im_w, im_h))

        temp = img[k] - img[k].min()
        temp = temp / temp.max()
        temp = temp * 255
        temp = temp.cpu().numpy().astype(np.uint8)
        
        print(temp.max(), temp.min())

        blur = cv2.GaussianBlur(temp,(5,5),0)
        ret3, thres3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        T = int(ret3)
        P = temp[insert_order[0][1], insert_order[0][0]]


# plt.subplot(17,3,cnt)
# img = (1/(2*2.8)) * image[0].permute(1,2,0).cpu().numpy() + 0.5
# img = functional.to_pil_image(img , mode="RGB")
# plt.imshow(img)
# plt.show()